### importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate, GlobalAveragePooling2D, BatchNormalization
output_size = 1

### 

In [ ]:
# data preprocessing

train_dir="F:/khanh_covid/split_data/train"
val_dir='F:/khanh_covid/split_data/val'
test_dir='F:/khanh_covid/split_data/test'

train_datagen2=ImageDataGenerator(rescale=1.0/255,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  rotation_range = 20,
                                  zca_whitening = True,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  zoom_range = 0.2,
                                 )

val_datagen2=ImageDataGenerator(rescale=1.0/255,
                                )

test_datagen2=ImageDataGenerator(rescale=1.0/255,
                                 )

train_generator2=train_datagen2.flow_from_directory(train_dir,target_size=(224,224),batch_size=128,class_mode='binary')

val_generator2=val_datagen2.flow_from_directory(val_dir,target_size=(224,224),batch_size=128,class_mode='binary')

test_generator2=test_datagen2.flow_from_directory(test_dir,target_size=(224,224),batch_size=128,class_mode='binary')



In [ ]:
print(train_generator2.class_indices)

### VGG16 Model


In [ ]:
# vgg16 modify
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

freezing = True
if freezing:
    for layer in base_model.layers[:-4]:
        layer.trainable = False
   
    for layer in base_model.layers[-4:]:
        print(layer.name)
        layer.trainable = True
        
# output layers

model = Flatten()(base_model.output)
model = Dropout(0.3)(model)
model = Dense(1024, activation='relu')(model)
model = Dropout(0.25)(model)
model = Dense(output_size, activation='sigmoid')(model)

model = tf.keras.Model(inputs = base_model.input, outputs = model)

loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
optimizer = Adam(learning_rate=5e-6, amsgrad=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Print model summary
# model.summary()


In [ ]:
# train
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Tạo callback ModelCheckpoint để lưu model tại epochs tốt nhất của val_loss
checkpoint = ModelCheckpoint('F:/khanh_covid/model/vgg16_modify_last_state.keras', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = model.fit(train_generator2,
          validation_data=val_generator2,
          epochs=40,     
          callbacks=[early_stopping, checkpoint])


with open('F:/khanh_covid/model/vgg16_modify_last_state.pkl', 'wb') as file:
    pickle.dump(history.history, file)

vgg16 = tf.keras.models.load_model('F:/khanh_covid/model/vgg16_modify_last_state.keras')
eval_result = vgg16.evaluate(test_generator2, steps=624)
eval_result

### DenseNet121 Model

In [ ]:
#  DenNet101 model
# load Densnet121 transfer learning
base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

# freeze all denselayers of the blocks except denselayer15, 16 of block4
freezing_layer = True
if freezing_layer:
    for layer in base_model.layers[:-114]:
        layer.trainable = False
     
    for layer in base_model.layers[-114:]:
        # print(layer.name)
        layer.trainable = True

# output layers
model = layers.GlobalAveragePooling2D()(base_model.output)
model = layers.Dropout(0.25)(model)
model = layers.Dense(output_size, activation='sigmoid')(model)

model = tf.keras.Model(inputs = base_model.input, outputs = model)

# loss
loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
optimizer = Adam(learning_rate=5e-6, amsgrad=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Print model summary
model.summary()

In [ ]:
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Tạo callback ModelCheckpoint để lưu model tại epochs tốt nhất của val_loss
checkpoint = ModelCheckpoint('F:/khanh_covid/model/dense_net121.keras', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = model.fit(train_generator2,
          validation_data=val_generator2,
          epochs=40,     
          callbacks=[early_stopping, checkpoint])


with open('F:/khanh_covid/model/dense_net121.pkl', 'wb') as file:
    pickle.dump(history.history, file)

dense_net121 = tf.keras.models.load_model('F:/khanh_covid/model/dense_net121.keras')
eval_result = dense_net121.evaluate(test_generator2, steps=624)
eval_result